In [ ]:
pip install -U crawl4ai

In [ ]:
# Install the package
!pip install -U crawl4ai

# (Optional) Install the pre-release version
!pip install crawl4ai --pre

# Run post-installation setup
!crawl4ai-setup

# Verify your installation
!crawl4ai-doctor


In [ ]:
!crawl4ai-setup

In [ ]:
!pip install nest_asyncio


In [64]:
import asyncio
import json
from pathlib import Path
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

async def main():
    browser_conf = BrowserConfig(headless=True)

    # 1) Extract article URLs from category page
    cat_schema = {
        "name": "articles",
        "baseSelector": "li.wp-block-post div.loop-card__content",
        "fields": [
            {
                "name": "category",
                "selector": ".loop-card__cat-group .loop-card__cat",
                "type": "text"
            },
            {
                "name": "url",
                "selector": "h3.loop-card__title a.loop-card__title-link",
                "type": "attribute",
                "attribute": "href"
            },
            {
                "name": "headline",
                "selector": "h3.loop-card__title a.loop-card__title-link",
                "type": "text"
            },
            {
                "name": "author",
                "selector": ".loop-card__meta .loop-card__author",
                "type": "text"
            },
            {
                "name": "date",
                "selector": ".loop-card__meta time",
                "type": "attribute",
                "attribute": "datetime"
            },
            {
                "name": "image_src",
                "selector": "figure.loop-card__figure img",
                "type": "attribute",
                "attribute": "src"
            }
        ]
    }
    cat_run_conf = CrawlerRunConfig(
        cache_mode = CacheMode.BYPASS,
        extraction_strategy = JsonCssExtractionStrategy(cat_schema)
    )

    async with AsyncWebCrawler(config=browser_conf) as crawler:
        result_cat = await crawler.arun(
            url = "https://techcrunch.com/category/startups/",
            config = cat_run_conf
        )

    if not result_cat.success:
        print("Category crawl failed:", result_cat.error_message)
        return

    article_list = json.loads(result_cat.extracted_content)
    print(f"Found {len(article_list)} articles")
    for article in article_list:
     url = article.get('url')
     if url:
         print(url)
     else:
         print("Skipping an article with a missing URL.")

    # Setup Markdown Generator for the article pages
    md_generator = DefaultMarkdownGenerator()

    # Directory to save markdown files
    out_dir = Path("articles_md")
    out_dir.mkdir(exist_ok=True)

    # 2) For each article URL → crawl and save markdown
    async with AsyncWebCrawler(config=browser_conf) as crawler:
        for ai in article_list:
            article_url = ai.get("url")
            if not article_url or not article_url.startswith(("http://", "https://")):
                print("Skipping invalid URL:", article_url)
                continue

            run_conf = CrawlerRunConfig(
                cache_mode = CacheMode.BYPASS,
                markdown_generator = md_generator
            )

            result_art = await crawler.arun(
                url = article_url,
                config = run_conf
            )

            if not result_art.success:
                print("Failed to crawl article:", article_url, "error:", result_art.error_message)
                continue

            md = result_art.markdown.raw_markdown
            # File name from URL (last segment)
            filename = article_url.rstrip("/").split("/")[-1] + ".md"
            filepath = out_dir / filename
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(md)
            print("Saved:", filepath)

if __name__ == "__main__":
    await main()
    


[INIT].... → Crawl4AI 0.7.5 

[FETCH]... ↓ https://techcrunch.com/category/startups/                                                            |
✓ | ⏱: 0.53s 

[SCRAPE].. ◆ https://techcrunch.com/category/startups/                                                            |
✓ | ⏱: 0.03s 

[EXTRACT]. ■ https://techcrunch.com/category/startups/                                                            |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://techcrunch.com/category/startups/                                                            |
✓ | ⏱: 0.60s 

...

Saved: articles_md/indias-kuku-snags-85m-as-mobile-content-wars-intensify.md


[FETCH]... ↓ https://techcrunch.com/2025/10/15/electric-aircr...up-beta-technologies-seeks-to-raise-825m-in-ipo/  |
✓ | ⏱: 0.60s 

[SCRAPE].. ◆ https://techcrunch.com/2025/10/15/electric-aircr...up-beta-technologies-seeks-to-raise-825m-in-ipo/  |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://techcrunch.com/2025/10/15/electric-aircr...up-beta-technologies-seeks-to-raise-825m-in-ipo/  |
✓ | ⏱: 0.63s 

Saved: articles_md/electric-aircraft-startup-beta-technologies-seeks-to-raise-825m-in-ipo.md
Skipping invalid URL: None
Skipping invalid URL: None
Skipping invalid URL: None
Skipping invalid URL: None
Skipping invalid URL: None
Skipping invalid URL: None
Skipping invalid URL: None
